In [1]:
import pandas as pd
import numpy as np

There are two type of biases:
-Sample weigths deviated from population means: It can be corrected with census data
-Something that prevents people form voting: 

# Data

In [316]:
users = pd.read_csv('sample_data/users_table.csv')
votes = pd.read_csv('sample_data/votes_table.csv')
census = pd.read_csv('sample_data/suffolk.csv')

In [317]:
census = census.dropna()

# Formato

### Census

Cuadrar la información del censo con lo recolectado

In [318]:
#Calcular las variables para el caso del censo
census = census.copy()
census['adj_factor'] = census['A_FNLWGT'] / 100

In [319]:
# Generar rangos de variables
census['age'] = pd.cut(census['A_AGE'], bins=[-1,21,25,35,45,60,1000],
                           labels=[0,1,2,3,4,5]).astype('int')
#Eliminate people under 21
census = census[census['age']!=0]
census['sex'] = np.where(census['A_SEX']==1,0,1)

In [320]:
#0=single, 1=married, 2=divorced
census.loc[census['A_MARITL']<4, 'marital_status'] = 1
census.loc[census['A_MARITL']==7, 'marital_status'] = 0
census.loc[census['A_MARITL']==5, 'marital_status'] = 2
census['marital_status'] = census['marital_status'].fillna(3)

In [321]:
#0=White, 1=Black, 2=Latino Esta variable no incluye Latinos
census.loc[census['PRDTRACE']==1, 'ethnicity'] = 0
census.loc[census['PRDTRACE']==2, 'ethnicity'] = 1
census['ethnicity'] = census['ethnicity'].fillna(3)

In [322]:
#Monthly income (1000,10000)
census['PEARNVAL_monthly'] = census['PEARNVAL']/12
census['income'] = pd.cut(census['PEARNVAL_monthly'], bins=[-1,1000,2500,5000,7000,1000000],
                           labels=[0,1,2,3,4]).astype('int')

In [323]:
# Agrupar observaciones del censo
census_grouped = census.groupby(['region_id', 'age', 'sex',
                                 'marital_status', 'ethnicity',
                                 'income'
                                ]).agg({'adj_factor':'sum'}).reset_index()

In [324]:
census_grouped['adj_factor'].sum()

1032249.1200000001

In [325]:
#This can be done only if you have only one region
#census_grouped['weight_census'] = census_grouped['adj_factor'] / census_grouped['adj_factor'].sum()

In [326]:
#census_grouped = census_grouped.drop(columns=['adj_factor'])

In [327]:
census_grouped['marital_status'] = census_grouped['marital_status'].astype('int')
census_grouped['ethnicity'] = census_grouped['ethnicity'].astype('int')

In [328]:
census_grouped.head()

,region_id,age,sex,marital_status,ethnicity,income,adj_factor
0,1,1,0,0,0,0,4431.16
1,1,1,0,0,0,1,8898.50
2,1,1,0,0,0,2,4129.49
3,1,1,0,0,1,1,18217.76
4,1,1,0,0,3,1,3947.46


### Votes

In [329]:
votes_users = votes.copy()
votes_users = votes_users.merge(users, how='left', on='user_id')

In [330]:
#Age
votes_users['age'] = pd.cut(votes_users['age'], bins=[-1,21,25,35,45,60,1000],
                           labels=[0,1,2,3,4,5]).astype('int')
votes_users = votes_users[votes_users['age']!=0]
#Income
votes_users['income'] = pd.cut(votes_users['income'], bins=[-1,1000,2500,5000,7000,1000000],
                           labels=[0,1,2,3,4]).astype('int')

In [331]:
votes_grouped = votes_users.groupby(['region_id', 'issue', 'vote',
                                     'sex', 'age', 'marital_status',
                                     'income','ethnicity'
                                    ]).agg({'user_id':'count'}).reset_index()

In [332]:
votes_grouped = votes_grouped.rename(columns={'user_id':'votes'})

In [334]:
# Select one region and one issue
# One issue, one region
df = votes_grouped.copy()
df = df[(df['issue']==1)&(df['region_id']==1)]

In [335]:
df_2 = pd.pivot_table(df, values='votes', index=['region_id', 'issue', 'sex',
                                     'age', 'marital_status',
                                     'income','ethnicity'],
                    columns=['vote'], aggfunc=np.sum).reset_index()

In [336]:
df_2 = df_2.rename(columns={0:'votes_against',
                            1:'votes_in_favor',
                           })

In [337]:
df_2['votes_in_favor'] = df_2['votes_in_favor'].fillna(0)
df_2['votes_against'] = df_2['votes_against'].fillna(0)

In [338]:
df_2['total_votes'] = df_2['votes_against']+df_2['votes_in_favor']

In [339]:
df_2['sample_weight'] = df_2['total_votes']/df_2['total_votes'].sum()

In [340]:
df_2.head()

vote,region_id,issue,sex,age,marital_status,income,ethnicity,votes_against,votes_in_favor,total_votes,sample_weight
0,1,1,0,1,0,2,0,5.0,3.0,8.0,0.008056
1,1,1,0,1,0,2,1,0.0,3.0,3.0,0.003021
2,1,1,0,1,0,2,2,5.0,2.0,7.0,0.007049
3,1,1,0,1,0,3,0,4.0,2.0,6.0,0.006042
4,1,1,0,1,0,4,2,4.0,4.0,8.0,0.008056


# Join

In [341]:
#Be cautious with duplicates. Like with the vote column (it duplicates the same entry twice)

In [342]:
df_final = df_2.merge(census_grouped, how='left',
                   on=['region_id', 'age', 'sex', 
                       'marital_status', 'ethnicity','income'
                      ])

In [344]:
df_final['adj_factor'] = df_final['adj_factor'].fillna(0)

In [345]:
df_final['weight_census'] = df_final['adj_factor']/df_final['adj_factor'].sum()

In [346]:
#Esto hay que verificarlo, no estoy seguro
df_final['weight_census'] = df_final['weight_census'].fillna(0)

In [347]:
df_final['adj_factor'].sum()

202404.18

In [348]:
df_final['weight_census'].sum()

1.0

# Estimates

In [349]:
df_final['perc_favor'] = df_final['votes_in_favor']/df_final['total_votes']
df_final['perc_against'] = df_final['votes_against']/df_final['total_votes']

In [350]:
df_final['perc_favor_sample'] = df_final['perc_favor']*df_final['sample_weight']
df_final['perc_against_sample'] = df_final['perc_against']*df_final['sample_weight']

In [351]:
df_final['perc_favor_adj'] = df_final['perc_favor']*df_final['weight_census']
df_final['perc_against_adj'] = df_final['perc_against']*df_final['weight_census']

In [357]:
df_final.groupby('age').agg({'perc_favor_sample':'sum',
                             'perc_against_sample':'sum',
                             'perc_favor_adj':'sum',
                             'perc_against_adj':'sum',
                            })#['perc_against_adj'].sum()

,perc_favor_sample,perc_against_sample,perc_favor_adj,perc_against_adj
age,,,,
1,0.044310,0.044310,0.007651,0.012751
2,0.065458,0.069486,0.034783,0.068182
3,0.114804,0.121853,0.211273,0.222931
4,0.149043,0.150050,0.082863,0.250298
5,0.109768,0.130916,0.053199,0.056068
